In [ ]:
import re
import sys
import csv
import pandas as pd

PATHS:

In [ ]:
#hate speech:
HATE_RAW_PATH = '../datasets/hate/raw/'
HATE_INTERIM_PATH = '../datasets/hate/interim/'

#emoji:
EMOJI_RAW_PATH = '../datasets/emoji/raw/'
EMOJI_INTERIM_PATH = '../datasets/emoji/interim/'

FILES:

In [ ]:
HATE_TEST = 'test_text.txt'
HATE_INTERIM_TEST = 'seperated.csv'

EMOJI_TEST = 'test_text.txt'
EMOJI_INTERIM_TEST = 'seperated.csv'

FUNCTIONS:

In [ ]:
def tokenise(line):
    # Initialise lists
    tokens = []
    unmatchable = []

    # Compile patterns for speedup
    token_pat = re.compile(r'\w+|#|\'|@')
    skippable_pat = re.compile(r'[.,]+')  # typically spaces

    # As long as there's any material left...
    while line:
        # Try finding a skippable token delimiter first.
        skippable_match = re.search(skippable_pat, line)
        if skippable_match and skippable_match.start() == 0:
            # If there is one at the beginning of the line, just skip it.
            line = line[skippable_match.end():]
        else:
            # Else try finding a real token.
            token_match = re.search(token_pat, line)
            #print(token_match)
            if token_match and token_match.start() == 0:
                #print(line[token_match.start():token_match.end()])
                if line[token_match.start():token_match.end()] == '#': #keep hash tags together and seperate
                    try:
                        token_match2 = re.search(token_pat, line[1:])
                        tokens.append(line[:token_match2.end()+1])
                        line = line[token_match2.end()+1:]
                    except:
                        line = line[token_match.end():]

                elif line[token_match.start():token_match.end()] == '@': #keep @ tags together and seperate
                    try:
                        token_match2 = re.search(token_pat, line[1:])
                        tokens.append(line[:token_match2.end()+1])
                        line = line[token_match2.end()+1:]
                    except:
                        line = line[token_match.end():]

                elif line[token_match.start():token_match.end()] == "'": #handle contractions as a single word
                    try:
                        token_match2 = re.search(token_pat, line[1:])
                        if ' ' in line[token_match2.start():token_match2.end()]:
                            line = line[token_match.end():]
                        
                        else: 
                            tokens.append(line[:token_match2.end()+1])
                            line = line[token_match2.end()+1:]
                    except:
                        line = line[token_match.end():]

                # If there is one at the beginning of the line, tokenise it.
                else:
                    tokens.append(line[:token_match.end()])
                    line = line[token_match.end():]
            else:
                # Else there is unmatchable material here.
                # It ends where a skippable or token match starts, or at the end of the line.
                unmatchable_end = len(line)
                if skippable_match:
                    unmatchable_end = skippable_match.start()
                if token_match:
                    unmatchable_end = min(unmatchable_end, token_match.start())
                # Add it to unmatchable and discard from line.
                unmatchable.append(line[:unmatchable_end])
                line = line[unmatchable_end:]

    final_tokens = []

    while len(tokens) > 0:
        temp1 = tokens.pop(0)
        try:
            temp2 = tokens.pop(0)
            if temp2[0] == "'":
                temp1 += temp2
                final_tokens.insert(0, temp1)
                #print('a', temp1)
            else:
                final_tokens.insert(0, temp1)
                tokens.insert(0, temp2)
                #print('b', temp1, temp2)
        except:
            final_tokens.insert(0, temp1)
            #print('d', temp1)
        
    final_tokens = final_tokens[::-1]

    #print(final_tokens)
    #print(unmatchable)
    return final_tokens

In [ ]:
f = open(HATE_RAW_PATH + HATE_TEST, "r", encoding="utf-8")
final_hate = []
for line in f:
    final_hate.append(tokenise(line))
f.close()

with open(HATE_INTERIM_PATH+HATE_INTERIM_TEST, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerows(final_hate)

In [ ]:
final_hate[0]

In [ ]:
f = open(HATE_RAW_PATH + HATE_TEST, "r", encoding="utf-8")
final_emoji = []
for line in f:
    final_emoji.append(tokenise(line))
f.close()

with open(EMOJI_INTERIM_PATH+EMOJI_INTERIM_TEST, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerows(final_emoji)

In [ ]:
final_emoji[0]